In [32]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [34]:
Keyword = pd.read_csv('./Keyword.csv')
Keyword

,0
0,서울특별시 종로구 삼일대로 382 상해
1,서울특별시 종로구 창신길 78 짱구
2,서울특별시 종로구 종로 385 교촌치킨 숭인점
3,서울특별시 종로구 새문안로5길 37 수제비집
4,서울특별시 종로구 세종대로23길 54 아람
...,...
668,서울특별시 종로구 동숭1길 4 속이 알찬 주먹밥
669,서울특별시 종로구 낙산길 196 좋아좋아분식
670,서울특별시 종로구 창경궁로34길 18-4 헬로사이공
671,서울특별시 종로구 종로 318 호랭이 뼈다귀


In [47]:
# 창이 나오지 않게 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'chromedriver.exe' # 크롬드라이버 경로 설정

source_url = "https://map.kakao.com/" # 리뷰주소 크롤링을 해 올 주소 설정

page_urls = [] #리뷰주소 크롤링 결과를 담을 리스트

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for k in tqdm_notebook(Keyword['0']):
    try:
        driver.get(source_url)

        # 검색창 찾기
        search_bar = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
        search_bar.send_keys(k)
        
        # 검색창 찾기
        searchbutton = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.submit")
        driver.execute_script('arguments[0].click();', searchbutton)

        driver.implicitly_wait(10)

        # 상세보기에 링크된 리뷰주소 가져오기
        moreview = driver.find_element(By.CLASS_NAME, 'moreview')
        page_url = moreview.get_attribute("href")
        page_urls.append(page_url)
    except:
        pass

driver.quit()

page_urls

  0%|          | 0/673 [00:00<?, ?it/s]

['https://place.map.kakao.com/13333296',
 'https://place.map.kakao.com/8211238',
 'https://place.map.kakao.com/521777293',
 'https://place.map.kakao.com/1636587272',
 'https://place.map.kakao.com/1843000947',
 'https://place.map.kakao.com/21233666',
 'https://place.map.kakao.com/1009998933',
 'https://place.map.kakao.com/21233647',
 'https://place.map.kakao.com/11143857',
 'https://place.map.kakao.com/21327011',
 'https://place.map.kakao.com/1148538280',
 'https://place.map.kakao.com/8019240',
 'https://place.map.kakao.com/20537009',
 'https://place.map.kakao.com/15828552',
 'https://place.map.kakao.com/15623350',
 'https://place.map.kakao.com/21407985',
 'https://place.map.kakao.com/19622361',
 'https://place.map.kakao.com/10377465',
 'https://place.map.kakao.com/9286484',
 'https://place.map.kakao.com/20616390',
 'https://place.map.kakao.com/1533086984',
 'https://place.map.kakao.com/21408916',
 'https://place.map.kakao.com/9787700',
 'https://place.map.kakao.com/141674389',
 'https:

In [ ]:
# 이미 리뷰주소가 있을 경우 사용
'''page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver' '''

In [49]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        driver.implicitly_wait(10)

        name = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2')
        name = name.text
        star = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span')
        star = ''.join([i for i in star.get_attribute("style") if i.isdigit()])
        review = driver.find_elements(By.CLASS_NAME, 'txt_comment')
        for i in review:
            review_df.loc[len(review_df)] = [name, star, i.text]

    except:
        # 리뷰가 없거나 비공개인 경우
        pass
    
driver.quit()

review_df

  0%|          | 0/369 [00:00<?, ?it/s]

,name,score,review
0,상해,20,식탁 바로 위 다운라이트 조명에 날벌레 가득 \n에어컨 바람에 날벌레 떨어질 가능성...
1,상해,20,별로예요\n최악까진 아니지만 짜장면에 기름이 너무 많아서 중간부턴 먹기 힘들었습니다...
2,상해,20,숙소와 가까웠지만 맛은 진짜 별로네요ㅠㅠ
3,교촌치킨 숭인점,100,맛있어요.
4,교촌치킨 숭인점,100,다른 지점보다 기름에 살짝 더 많이 절여진 맛이에요..
...,...,...,...
717,타미하우스,100,아침에 모닝세트로 가볍게 한끼하기에 좋아요! \n샌드위치 종류도 3~4개정도 되고 ...
718,타미하우스,100,샌드위치 진짜 맛있어요 제가 먹은 샌드위치중에 탑이예요 ~ 직원분이 게살샌드위치가 ...
719,타미하우스,100,
720,화순집,80,"23,000원- 곱창볶음 공기밥1 볶음밥1"


In [50]:
# 리뷰 없이 별점만 남긴 리뷰 제거
review_df = review_df[review_df['review'] != '']
review_df

,name,score,review
0,상해,20,식탁 바로 위 다운라이트 조명에 날벌레 가득 \n에어컨 바람에 날벌레 떨어질 가능성...
1,상해,20,별로예요\n최악까진 아니지만 짜장면에 기름이 너무 많아서 중간부턴 먹기 힘들었습니다...
2,상해,20,숙소와 가까웠지만 맛은 진짜 별로네요ㅠㅠ
3,교촌치킨 숭인점,100,맛있어요.
4,교촌치킨 숭인점,100,다른 지점보다 기름에 살짝 더 많이 절여진 맛이에요..
...,...,...,...
716,웰빙김밥,100,맛있고 양이 대박입니다!!!\n참치김밥 1줄 다먹음 배 엄청 불러요
717,타미하우스,100,아침에 모닝세트로 가볍게 한끼하기에 좋아요! \n샌드위치 종류도 3~4개정도 되고 ...
718,타미하우스,100,샌드위치 진짜 맛있어요 제가 먹은 샌드위치중에 탑이예요 ~ 직원분이 게살샌드위치가 ...
720,화순집,80,"23,000원- 곱창볶음 공기밥1 볶음밥1"


In [51]:
review_df.to_csv("./review_data.csv", index=False)